In [1]:
import os
import json
import requests
import base64

import numpy as np
import pandas as pd


In [2]:
base_dir =os.path.expanduser('~')
home_dir =f'{base_dir}/jpx'


home_dir

'/home/jupyter/jpx'

In [3]:
import sys
import datetime

sys.path.append(home_dir)
sys.path.append(home_dir + '/predict/src')
from predictor import ScoringService
#from fundamental.predictor import ScoringServiceFundamental



import importlib
#import api
#importlib.reload(api.fetch_api)
from api.fetch_api import (
    fetch_jquant_data,
    call_refresh_api,
    call_jquants_api,
    match_type_df
)
from api import token

refreshtoken = token.refreshtoken
idtk = call_refresh_api(refreshtoken)['idToken']



In [4]:
DATASET_DIR=f'{home_dir}/data'
MODEL_PATH=f'{home_dir}/predict/model'


inputs = ScoringService.get_inputs(DATASET_DIR)


load_data = ['stock_list', 'stock_price', 'stock_fin', 'stock_labels', 'nikkei_article','tdnet' ]

dfs = ScoringService.get_dataset(inputs, load_data)

codes = dfs['stock_list']['Local Code'].unique() #全銘柄

INFO : 2021-05-08 10:16:39,147 : read stock_list
INFO : 2021-05-08 10:16:39,170 : read stock_price
INFO : 2021-05-08 10:16:50,176 : read stock_fin
/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
INFO : 2021-05-08 10:16:50,769 : read tdnet
INFO : 2021-05-08 10:16:53,102 : read nikkei_article
INFO : 2021-05-08 10:16:55,314 : read stock_labels


In [5]:
publish_datetime_list = dfs['nikkei_article']['publish_datetime'].unique()
publish_datetime_list[0]

'2020-01-01T00:00:00+09:00'

In [6]:
df_nikkei_article = dfs['nikkei_article']
df_nikkei_article['publish_datetime'] = pd.to_datetime(df_nikkei_article['publish_datetime'])
df_nikkei_article['date'] = df_nikkei_article['publish_datetime'].apply(lambda x:x.date())

In [7]:
df_nikkei_article['date'].iloc[0]

datetime.date(2020, 1, 1)

In [8]:
aaa = df_nikkei_article.groupby('date').apply(lambda x:len(x))

In [9]:
aaa.sort_values()

date
2020-03-15      97
2020-06-14     102
2020-04-12     102
2020-01-02     104
2020-09-13     108
              ... 
2020-01-31     875
2020-05-15     881
2020-05-14     882
2020-03-31     931
2020-02-28    1003
Length: 366, dtype: int64

In [10]:
start = '2021-04-01'
end = '2021-05-07'
paramdict = {}
news = []
datefrom = start
while True:
    if datefrom == end:
        break
    
    paramdict['datefrom'] = datefrom
    paramdict["dateto"] = datefrom
    print( paramdict['datefrom'])
    res = call_jquants_api(paramdict, idtk, "news")
    df_news = pd.DataFrame(res['news'])
    datefrom =  paramdict["dateto"] 
    news.append(df_news)
    print(len(df_news))
    while len(df_news)==250:
        
        paramdict.clear()
        paramdict['scrollId'] = res['scrollId']
        res2 = call_jquants_api(paramdict, idtk, "news")
        df_news = pd.DataFrame(res2['news'])
        print(len(df_news))
        news.append(df_news)
    paramdict.clear()
    datefrom = (pd.to_datetime(datefrom).date() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    
    
#     # save
#     try:
#         news += res['news']
#     except:
#         continue # データ取得に失敗すると、{'message': 'Internal server error'}になる

2021-04-01
250
250
196
2021-04-02
250
250
85
2021-04-03
180
2021-04-04
184
2021-04-05
250
200
2021-04-06
250
247
2021-04-07
250
250
10
2021-04-08
250
250
34
2021-04-09
250
244
2021-04-10
203
2021-04-11
149
2021-04-12
250
190
2021-04-13
250
250
15
2021-04-14
250
250
57
2021-04-15
250
250
23
2021-04-16
250
250
8
2021-04-17
234
2021-04-18
145
2021-04-19
250
250
15
2021-04-20
250
250
40
2021-04-21
250
250
46
2021-04-22
250
250
64
2021-04-23
250
250
52
2021-04-24
204
2021-04-25
164
2021-04-26
0
2021-04-27
0
2021-04-28
0
2021-04-29
0
2021-04-30
0
2021-05-01
0
2021-05-02
0
2021-05-03
0
2021-05-04
0
2021-05-05
0
2021-05-06
0


In [11]:
df_news_20210405 = pd.concat(news)

In [13]:
df_news_20210405.rename(columns={'stock_code' : 'company_g.stock_code'})

,article_id,publish_datetime,media_code,media_name,men_name,headline,keywords,classifications,company_g.stock_code
0,TDSKDBDGXZQODK314J2031032021000000,2021-04-01T00:00:00Z,TNY,日本経済新聞電子版,,春秋（4月1日）,春秋\n福沢諭吉\n防衛大学,「春秋」\n＃Ｂ０６３０,
1,TDSKDBDGXZQOGH309X3030032021000000,2021-04-01T00:00:00Z,TNY,日本経済新聞電子版,,反物質の操作に世界初の成功 日本など国際研究グループ,グループ\n世界初\n反物質\n国際研究\n成功\n操作\n百瀬孝昌\n藤原真琴,＄絵写表記事\n＄テクノロジー\n＃Ｗ４０１０８,
2,TDSKDBDGXZQOH131B1Y031032021000000,2021-04-01T00:00:00Z,TNY,日本経済新聞電子版,,人事、厚生労働省,厚生労働省,＃Ｋ１\n＃Ｂ０６３０,
3,TDSKDBDGXZQOFS316NH031032021000000,2021-04-01T00:05:56Z,TNY,日本経済新聞電子版,,首相、総務相「職責果たしている」,政府\n田村憲久\n総務大臣\n総務省\n職責\n衆議院\n首相,＃Ｗ２０２０１\n＃Ｗ２０２０３\n＃Ｂ０６３０,
4,TDSKDBDGXZQOGN31DPW031032021000000,2021-04-01T00:35:58Z,TNY,日本経済新聞電子版,,3月のシカゴPMI、6.8ポイント上昇 2年8カ月ぶり高水準,シカゴＰＭＩ\n高水準,＃Ｗ３０２０１\n＃Ｗ３０２０７\n＃Ｋ７\n＃Ａ８５８\n＃Ａ９９４,
...,...,...,...,...,...,...,...,...,...
159,TDSKDBDGXZQODH250GO025042021000000,2021-04-25T21:21:42Z,TNY,日本経済新聞電子版,,星野唯一の4日間60台、新スイングで安定感 男子ゴルフ,ゴルフ\nスイング\n安定感\n星野唯一\n男子ゴルフ,＄絵写表記事\n＄スポーツ\n＄ゴルフ,
160,TDSKDBDGXZQOUA253O2025042021000000,2021-04-25T21:42:34Z,TNY,日本経済新聞電子版,,野党共闘、2選挙区で一定の効果 共同通信出口調査,野党共闘\n出口調査,＄通信社記事\n＄共同通信社記事\n＃Ｗ２０１０２\n＃Ｗ２０１０１,
161,TDSKDBDGXZQOFD157DY015042021000000,2021-04-25T22:42:10Z,TNY,日本経済新聞電子版,,名古屋市長選、現職の河村氏が当選,公明党\n名古屋\n大村秀章\n市長選\n当選\n河村たかし\n現職\n立憲民主党\n自民党,＃Ｗ２０１０１\n＃Ｗ２０１０３\n＃Ａ４２２\n＃Ａ９８５\n＃Ｂ０６３０,
162,TDSKDBDGXLSSXK00100_25042021000000,2021-04-25T23:02:15Z,TNY,日本経済新聞電子版,,カナダ、世界リレーの派遣断念 米国などに続き,ＩＡＡＦ世界リレー大会\nカナダ\nアメリカ合衆国\n派遣断念,＄通信社記事\n＄共同通信社記事,


In [14]:
df_news_add = df_news_20210405.rename(columns={'stock_code' : 'company_g.stock_code'})
#for i in ['picture_flag', 'paragraph_cnt', 'char_length', 'page_from']:
df_news_add['picture_flag'] = np.nan
df_news_add['paragraph_cnt'] = -1

df_news_add['char_length'] = -1
df_news_add['page_from'] = -1


In [17]:
del df_nikkei_article['date']


df_news_add_type = match_type_df(df_news_add, df_nikkei_article)

df_news_add_type.to_csv('news_20210405.csv.gz', compression='gzip', index=False)


In [19]:
df_news_1 = pd.read_csv('news_20210103.csv.gz')

In [20]:
df_news_main = pd.concat([dfs['nikkei_article'], df_news_1, df_news_add_type])

In [24]:


df_news_main.to_csv('nikkei_article.csv.gz', compression='gzip', index=False)


In [23]:
df_news = pd.DataFrame(res['news'])

,article_id,publish_datetime,media_code,media_name,men_name,headline,keywords,classifications,stock_code
0,TDSKDBDGXZQODG020SF002012021000000,2021-01-02T20:26:10Z,TNY,日本経済新聞電子版,,会いたいけれど…我慢の正月 オンライン帰省や分散初詣,オンライン\nオンライン帰省\n分散初詣\n帰省\n我慢\n正月,＄絵写表記事\n＄社会・くらし\n＄新型コロナウイルス,
1,TDSKDBDGXZQODE23AB3023122020000000,2021-01-02T21:04:53Z,TNY,日本経済新聞電子版,,日本からの入国規制、26カ国・地域で強化 感染再拡大で,入国規制\n再拡大\n地域\n安倍晋三\n感染\n浜田篤郎\n菅義偉\n規制,＄絵写表記事\n＄政治\n＄新型コロナウイルス\n＃Ｗ２０４０１\n＃Ｗ２０４０４,
2,TDSKDBDGXZQOGM0228Z002012021000000,2021-01-02T21:24:52Z,TNY,日本経済新聞電子版,,西アフリカのニジェール大統領選、2月に再投票,ニジェール政府\n西アフリカ\nニジェール,＄通信社記事\n＄共同通信社記事\n＄国際\n＄中東・アフリカ\n＃Ｗ２０１０３\n＃Ｗ２０...,
3,TDSKDBDGXLSSXK60585_02012021000000,2021-01-02T21:51:21Z,TNY,日本経済新聞電子版,,錦織、21年初戦はＡＴＰ杯 ２月１日開幕,ＡＴＰ\n初戦,＄通信社記事\n＄共同通信社記事,
4,TDSKDBDGXZQODE011FV001012021000000,2021-01-02T23:00:00Z,TNY,日本経済新聞電子版,,1・2日の菅首相の動静,動静\n厚生労働省\n政府\n菅\n衆議院\n首相,＄政治,
...,...,...,...,...,...,...,...,...,...
216,TDSKDBDGXZQODG020XH002012021000000,2021-01-02T16:25:59Z,TNY,日本経済新聞電子版,,NHK紅白の視聴率40.3% 初の無観客、前年より上昇,紅白\n視聴率,＄通信社記事\n＄共同通信社記事\n＃Ｗ５０５０３\n＃Ｗ５０５０１,
217,TDSKDBDGXZQOHD020KS002012021000000,2021-01-02T18:50:00Z,TNY,日本経済新聞電子版,,大阪の百貨店、初売り寂し 行列は例年の半分以下,例年\n初売り\n大阪\n百貨店\n秋田拓士\n行列,＄絵写表記事\nＴ８２４４\nＴ８２４２\nＰＤ４５１\nＮ０００１６６３\nＮ０００１６６...,8244\n8242
218,TDSKDBDGXZQODK286K8028122020000000,2021-01-02T19:00:00Z,TNY,日本経済新聞電子版,,［社説］「政治主導の時代」を踏まえた選択を,大学教授\n宇野重規\n安倍晋三\n政治主導\n時代\n社説\n菅義偉\n選択,＃Ｗ２０１０１\n＃Ｗ２０１０３\n＃Ｗ２０２０１\n＃Ｗ２０２０３\n＃Ｋ４,
219,TDSKDBDGXZQODG021L2002012021000000,2021-01-02T19:30:52Z,TNY,日本経済新聞電子版,,Uターン、人影まばら 交通機関は混乱なく,交通機関\nＵターン\n人影,＄通信社記事\n＄共同通信社記事,


In [25]:
res2

{'news': [],
 'scrollId': 'FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldGNoBRRfMXdXUzNrQkNSalFKN2FoN0hKNwAAAAAAAOJCFkd4Qm5xUGs3Uk5DTU94MXI1Ulk1UUEUQUZ3V1Mza0JDUmpRSjdhaDdITjcAAAAAAADiQxZHeEJucVBrN1JOQ01PeDFyNVJZNVFBFEFsd1dTM2tCQ1JqUUo3YWg3SE43AAAAAAAA4kUWR3hCbnFQazdSTkNNT3gxcjVSWTVRQRRBMXdXUzNrQkNSalFKN2FoN0hOOAAAAAAAAOJGFkd4Qm5xUGs3Uk5DTU94MXI1Ulk1UUEUQVZ3V1Mza0JDUmpRSjdhaDdITjcAAAAAAADiRBZHeEJucVBrN1JOQ01PeDFyNVJZNVFB'}

In [13]:
scroll_id = df_news['scrollId'].iloc[0]
paramdict = {}
paramdict["scrollID"] = scroll_id
news_data = call_jquants_api(paramdict, idtk, "news")


In [14]:
df_news = pd.DataFrame(news_data)


In [15]:
df_news

,news,scrollId
0,{'article_id': 'TDSKDBDGXLASFL01H7E_0110202000...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...
1,{'article_id': 'TDSKDBDGXMZO6325291031082020TC...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...
2,{'article_id': 'TDSKDBDGXMZO5515989002022020CR...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...
3,{'article_id': 'TDSKDBDGXLASS0IMB01_0110202000...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...
4,{'article_id': 'TDSKDBDGXMZO6212765031072020TC...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...
...,...,...
245,{'article_id': 'TDSKDBDGXMZO551575300202202000...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...
246,{'article_id': 'TDSKDBDGXMZO6441799030092020X1...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...
247,{'article_id': 'TDSKDBDGXLSSXK00253_0111202000...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...
248,{'article_id': 'TDSKDBDGXMZO6441803030092020SH...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...


In [102]:
paramdict = {}
paramdict["datefrom"] = "2020-02-01"
paramdict["dateto"] = "2020-02-02"
#paramdict["code"] = "8697"
# paramdict["headline"] = " "
# paramdict["keyword"] = " "
news_data = call_jquants_api(paramdict, idtk, "news")

In [103]:
news_data

{'message': 'Internal server error'}

In [50]:

paramdict["scrollID"] = scroll_id


ccc = call_jquants_api(paramdict, idtk, "news")

In [52]:
ddd = pd.DataFrame(ccc)


In [53]:
ddd

,news,scrollId
0,{'article_id': 'TDSKDBDGXMZO551573900202202000...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...
1,{'article_id': 'TDSKDBDGXLSSXK00139_0202202000...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...
2,{'article_id': 'TDSKDBDGXMZO5515827002022020TJ...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...
3,{'article_id': 'TDSKDBDGXMZO5515854002022020FF...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...
4,{'article_id': 'TDSKDBDGXMZO5515883002022020FF...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...
...,...,...
245,{'article_id': 'TDSKDBDGXMZO5515296001022020EA...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...
246,{'article_id': 'TDSKDBDGXMZO5515436001022020EA...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...
247,{'article_id': 'TDSKDBDGXMZO5514351001022020EA...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...
248,{'article_id': 'TDSKDBDGXMZO5506426030012020EA...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...


In [45]:
paramdict

,news,scrollId
0,{'article_id': 'TDSKDBDGXMZO551573900202202000...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...
1,{'article_id': 'TDSKDBDGXLSSXK00139_0202202000...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...
2,{'article_id': 'TDSKDBDGXMZO5515827002022020TJ...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...
3,{'article_id': 'TDSKDBDGXMZO5515854002022020FF...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...
4,{'article_id': 'TDSKDBDGXMZO5515883002022020FF...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...
...,...,...
245,{'article_id': 'TDSKDBDGXMZO5515296001022020EA...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...
246,{'article_id': 'TDSKDBDGXMZO5515436001022020EA...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...
247,{'article_id': 'TDSKDBDGXMZO5514351001022020EA...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...
248,{'article_id': 'TDSKDBDGXMZO5506426030012020EA...,FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldG...


In [32]:
news_list = []
for i in codes:
    aaa = call_jquants_api(paramdict, idtk, "news")

pd.DataFrame(aaa['news'])

,article_id,publish_datetime,media_code,media_name,men_name,headline,keywords,classifications,stock_code
0,TDSKDBDGXMZO5515739002022020000000,2020-02-02T13:06:28Z,TNY,日本経済新聞電子版,,新型肺炎、フィリピンで死者 中国国外で初か\n中国本土は死者300人超、感染者1.4万人超,フィリピン\n死者\n新型肺炎\n中国国外\n中国本土,＄通信社記事\n＄共同通信社記事\n＃Ａ７１６\n＃Ａ７０７\n＃Ａ９９０,
1,TDSKDBDGXLSSXK00139_02022020000000,2020-02-02T14:00:31Z,TNY,日本経済新聞電子版,,車いすテニス、国枝慎吾が２年ぶりＶ 全豪オープン\n上地結衣はダブルス、シングルス２冠,国枝慎吾\n上地結衣\n車いすテニス\nテニス\n全豪オープン\nダブルス\nシングルス,＄通信社記事\n＄共同通信社記事,
2,TDSKDBDGXMZO5515827002022020TJ1000,2020-02-02T15:04:43Z,TNY,日本経済新聞電子版,,新型肺炎、マスクや除菌用品の品薄深刻に,大学教授\nクレベリン\nメルカリ\nドラッグストア\nココカラファイン\n大幸薬品\nユニ...,＄絵写表記事\nＴ３０９８\nＴ４５７４\nＴ８１１３\nＴ４４５２\nＰＤ４５４\nＰＤ０...,3098\n4574\n8113\n4452
3,TDSKDBDGXMZO5515854002022020FF8000,2020-02-02T16:48:07Z,TNY,日本経済新聞電子版,,ブルームバーグ氏が５兆ドル増税案 「富豪は税負担を」,米国政府\nマイケル・ブルームバーグ\nスーパーチューズデー\n元ニューヨーク市長\n市長\...,＃Ｗ２０１０３\n＃Ｗ２０１０１\n＃Ｗ２０３０４\n＃Ｗ２０３０１\n＃Ａ８５８\n＃Ａ９９４,
4,TDSKDBDGXMZO5515883002022020FF8000,2020-02-02T17:37:32Z,TNY,日本経済新聞電子版,,穏健か、変革か 米民主指名争いアイオワで始動へ\n「対トランプ」は誰か 大混戦に世界が固唾,米国政府\nマイケル・ムーア\nトランプ\nアイオワ州\n指名争い\n米国民主党\n初戦\n...,＃Ｗ２０１０３\n＃Ｗ２０１０１\n＃Ｗ２０４０１\n＃Ａ８５８\n＃Ａ９９４,
...,...,...,...,...,...,...,...,...,...
245,TDSKDBDGXMZO5515296001022020EA1000,2020-02-01T23:00:00Z,TNY,日本経済新聞電子版,,磁気テープ復権 企業データ50年以上保存、低コスト,磁気テープ\nＨＤＤ\nデータ\nビッグデータ\n低コスト\n復権\n産業\n保存,＄絵写表記事\nＴ４９０１\nＴ６７５８\nＰＤ０８１\nＰＤ２３３\nＮ００００６３８\n...,4901\n6758
246,TDSKDBDGXMZO5515436001022020EA5000,2020-02-01T23:00:00Z,TNY,日本経済新聞電子版,,インダストリー4.0「５割達成」 提唱者のカガーマン氏,ビジネスモデル\nインタビュー\nヘニング・カガーマン\nインターネット・オブ・シングス\n...,＄絵写表記事\n＃Ｋ６\n＃Ａ７７８\n＃Ａ９９１,
247,TDSKDBDGXMZO5514351001022020EA2000,2020-02-01T23:08:29Z,TNY,日本経済新聞電子版,,トランプ氏弾劾、無罪濃厚に ボルトン氏招致せず,米国政府\nミット・ロムニー\nスーザン・コリンズ\nラマー・アレクサンダー\nドナルド・ト...,＄絵写表記事\n＃Ｗ２０１０２\n＃Ｗ５０３０５\n＃Ｗ２０１０１\n＃Ｗ５０３０１\n＃Ａ...,
248,TDSKDBDGXMZO5506426030012020EA4000,2020-02-02T00:00:00Z,TNY,日本経済新聞電子版,,今週の予定 2月2～8日,,＄絵写表記事\nＴ８００１\nＴ７２７０\nＴ８００２\nＴ８０５８\nＴ４６８９\nＴ２８...,8001\n7270\n8002\n8058\n4689\n2801\n6758\n4502...


In [10]:
# df_news = pd.read_csv(f"{DATASET_DIR}/nikkei_article.csv.gz")
# df_news

In [11]:
dfs.keys()

dict_keys(['stock_list', 'stock_price', 'stock_fin', 'tdnet', 'nikkei_article', 'stock_labels'])

In [13]:
from tqdm import tqdm
df_stock_lists_list = []
for code in tqdm(codes):
    lists = pd.DataFrame(call_jquants_api(paramdict, idtk, "lists", f"{code}")['list'])
    df_stock_lists_list.append(lists)

100%|██████████| 3711/3711 [06:28<00:00,  9.54it/s]


In [14]:
stock_list = pd.concat(df_stock_lists_list)#.drop_duplicates()
#stock_list = stock_list[dfs['stock_list'].columns].reset_index(drop=True)

stock_list = match_type_df(stock_list, dfs['stock_list'])


In [15]:
stock_list_drop_col = stock_list.drop(['Effective Date', 'IssuedShareEquityQuote ModifyDate'], axis=1)


stock_list_api_list = []
for i in stock_list['Local Code'].unique():
    df_local = stock_list[stock_list['Local Code']==i]
    df_local_recent = df_local.sort_values(by='Effective Date', ascending=False).iloc[[0]]
    stock_list_api_list.append(df_local_recent)
    
stock_list_api = pd.concat(stock_list_api_list).reset_index(drop=True)


In [18]:
# stock_list更新分保存
#stock_list_api.to_csv('stock_list.csv.gz', compression='gzip', index=False)

In [ ]:
#%%capture output

df_stock_price_api = fetch_jquant_data(codes=codes, paramdict=paramdict, target="prices", idtk=idtk)


 35%|███▍      | 1283/3711 [21:07<41:36,  1.03s/it] 

In [74]:

df_stock_price_api = match_type_df(df_stock_price_api, dfs['stock_price'])

In [75]:
df_stock_price_api.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1056618 entries, 0 to 299
Data columns (total 16 columns):
 #   Column                                           Non-Null Count    Dtype  
---  ------                                           --------------    -----  
 0   Local Code                                       1056618 non-null  int64  
 1   EndOfDayQuote Date                               1056618 non-null  object 
 2   EndOfDayQuote Open                               1056618 non-null  float64
 3   EndOfDayQuote High                               1056618 non-null  float64
 4   EndOfDayQuote Low                                1056618 non-null  float64
 5   EndOfDayQuote Close                              1056618 non-null  float64
 6   EndOfDayQuote ExchangeOfficialClose              1056618 non-null  float64
 7   EndOfDayQuote Volume                             1056618 non-null  float64
 8   EndOfDayQuote CumulativeAdjustmentFactor         1056618 non-null  float64
 9   EndOfD

In [80]:
stock_price_concat = pd.concat([dfs['stock_price'].loc[:'2020-01-01'].reset_index(drop=True), df_stock_price_api])
#stock_price_concat.to_csv('stock_price.csv.gz', compression='gzip', index=False)

In [ ]:


stockfins_from_api = fetch_jquant_data(codes=codes, paramdict=paramdict, target="stockfins", idtk=idtk)





 64%|██████▍   | 2373/3711 [05:51<03:16,  6.82it/s]

In [83]:
stockfins_from_api = match_type_df(stockfins_from_api, dfs['stock_fin'])

In [85]:

stock_stockfins_concat = pd.concat([dfs['stock_fin'].loc[:'2020-01-01'].reset_index(drop=True), stockfins_from_api])
#stock_stockfins_concat.to_csv('stock_fin.csv.gz', compression='gzip', index=False)

In [89]:


stocklabels_from_api = fetch_jquant_data(codes=codes, paramdict=paramdict, target="stocklabels", idtk=idtk)



100%|██████████| 3523/3523 [39:06<00:00,  1.50it/s] 


In [94]:
stocklabels_from_api = match_type_df(stocklabels_from_api, dfs['stock_labels'])
stocklabels_from_api_concat = pd.concat([dfs['stock_labels'].loc[:'2020-01-01'].reset_index(drop=True), stocklabels_from_api])
stocklabels_from_api_concat.to_csv('stock_labels.csv.gz', compression='gzip', index=False)

In [92]:
stocklabels_from_api_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4394888 entries, 0 to 299
Data columns (total 11 columns):
 #   Column         Dtype  
---  ------         -----  
 0   base_date      object 
 1   Local Code     int64  
 2   label_date_5   object 
 3   label_high_5   float64
 4   label_low_5    float64
 5   label_date_10  object 
 6   label_high_10  float64
 7   label_low_10   float64
 8   label_date_20  object 
 9   label_high_20  float64
 10  label_low_20   float64
dtypes: float64(6), int64(1), object(4)
memory usage: 402.4+ MB
